<a href="https://colab.research.google.com/github/spyrgalaz/DNN/blob/main/ITML_OPTIMIZATION_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ITML OPTIMIZATION

### INSTALLATION PACKAGES

In [1]:
! pip install PuLP
! pip install XlsxWriter
! pip install xlrd

     |████████████████████████████████| 41.2 MB 81 kB/s 
     |████████████████████████████████| 148 kB 7.1 MB/s 


### IMPORT PACKAGES

In [2]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from pandas import datetime
import pulp as plp
from pulp  import *
import xlsxwriter
from openpyxl import Workbook

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  after removing the cwd from sys.path.


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io

### IMPORT DATA SHAPE DATA

In [ ]:
df1=pd.read_excel('ITML5.xlsx',sheet_name='PWI')
df=pd.read_excel('ITML5.xlsx',sheet_name='new_rate')
df3=pd.read_excel('ITML5.xlsx',sheet_name='FINAL MONTH')
df4=pd.read_excel('ITML5.xlsx',sheet_name='ΜΗΝΕΣ')

dfTHWP=pd.read_excel('ITML5.xlsx',sheet_name='CONSTRAINTS_WP')

dfWW_A=df1.loc[df1['ADMIN']=='A']


dfNNE=df.loc[df['COUNTRY_CODE']=='E']
dfNNC=df.loc[df['COUNTRY_CODE']=='C']
dfNNB=df.loc[df['COUNTRY_CODE']=='E/C']
dfNN_A=df.loc[df['ADMIN']=='A']


df=df.drop(columns=['COUNTRY_CODE','ADMIN'])
df1=df1.drop(columns=['ADMIN'])

dfNN=df.fillna(0)                ###### ΑΛΛΑΓΗ       

dfNN_A=dfNN_A.set_index(['NAME'])
dfWW_A=dfWW_A.set_index(['INDEX_PR_WP']) ##### ΑΛΛΑΓΗ ΓΙΑ  ADMIN
dfNNE=dfNNE.set_index(['NAME'])
dfNNC=dfNNC.set_index(['NAME'])
dfNNB=dfNNB.set_index(['NAME'])

dfWW=df1.set_index(['INDEX_PR_WP']) ####### ΔΕΙΚΤΕΣ

dfNN=dfNN.set_index(['NAME'])        ####### ΑΛΛΑΓΗ 

dfMM=df4.set_index(['ΜΗΝΕΣ'])


dfTHWP=dfTHWP.set_index('WP')

print(df3.head())
df3=pd.concat([df3]*39, ignore_index=False) ###### ΑΤΟΜΑ = 39
#df3=df3.set_index(['DATE'])
print(df3.head(49))                         ###### ΗΜΕΡΟΜΗΝΙΕΣ = 48(12Χ4)   
print(df3.shape)

df3=df3.melt( id_vars=['DATE'],var_name='PR/WP')
df3 = df3.sort_values(by = ['DATE','PR/WP'], ascending = [True,True])
df3=df3.reset_index(drop=True)
print(df3.head())
df3.isna().any()


print(df.shape)
df=df.fillna(0)

df=pd.concat([df]*86, ignore_index=True)     ########  ΕΡΓΑ = 106
df=df.melt( id_vars=['NAME'],var_name='DATE')
print(df.head(40))                              ####### ΓΙΑ 39 ΑΤΟΜΑ +1


df3['NAME']=df['NAME']
df3['RATE']=df['value']
df3['RATE_IN_WP']=df3['value']*df3['RATE']

df3.head(4)                                      ####### ΓΙΑ 39 ΑΤΟΜΑ +1

df3=df3[df3.RATE_IN_WP !=0]                       ####### ΑΛΛΑΓΗ

dftotal=df3.set_index(['DATE','NAME','PR/WP'])

print(dftotal.index)
print(dftotal.shape)


### CREATE INDEX LIST

In [ ]:
#p=dfkar.index.to_list()
n_A=dfNN_A.index.to_list()
w_A=dfWW_A.index.to_list()
t=dftotal.index.to_list()
w=dfTHWP.index.to_list()
n=dfNN.index.to_list()
nC=dfNNC.index.to_list()
nE=dfNNE.index.to_list()
nB=dfNNB.index.to_list()

In [ ]:
import time
start_time = time.clock()

### CREATE DECISION VARIABLES

In [ ]:
HOURS = plp.LpVariable.dicts("HOURS",
                                     ((i,j,k) for i, j, k in dftotal.index),
                                     lowBound=0,upBound=20,
                                     cat='Continuous')

In [ ]:
model = plp.LpProblem("hours_maximize", plp.LpMaximize)

### CREATE OBJECTIVE FUNCTION

In [ ]:
model += pulp.lpSum(
    [HOURS[i,j,k]*dftotal.loc[(i,j,k),'RATE_IN_WP'] for i, j, k in dftotal.index ]
)

### CONSTRAINTS FOR ADMIN

In [ ]:
for i in dfMM.index :
    for j in dfNN.index :
        if j in n_A :
            model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k  not in w_A if (i,j,k) in dftotal.index ])==0

In [ ]:
for i in dfMM.index :
    for j in dfNN.index :
        if j in n_A :
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index   for k in dfWW.index if k   in w_A if (i,j,k) in dftotal.index ])>=100

### CONSTRAINTS TOTAL REMAINING HOURS

In [ ]:
ho=0
for k in dfWW.index :
   if k in w :
     model += pulp.lpSum([HOURS[i,j,k] for i in dfMM.index  for j in dfNN.index if (i,j,k) in dftotal.index  ])==dfTHWP.loc[k,'TOT_HOURS_WP']
     ho=ho+1
ho  

### CONSTRAINTS FOR MANAGERS

In [ ]:
for i in dfMM.index :
    for k in dfWW.index[0:48] :
        if (i,'Μπράβος Γεώργιος',k) in dftotal.index :
            model +=HOURS[i,'Μπράβος Γεώργιος',k]>=1
    

In [ ]:
for i in dfMM.index :
    for k in dfWW.index[48:] :
        if (i,'Αλεξόπουλος Δημήτριος',k) in dftotal.index : 
            model +=HOURS[i,'Αλεξόπουλος Δημήτριος',k]>=1

### CONSTRAINTS FOR TOTAL HOURS PER WP

In [ ]:
for i in dfMM.index : ### GREECE - CYPRUS
    for j in dfNN.index :
        if j in nB:
            if j not in n_A :
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[0:48] if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_ HELLAS']
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[48:]  if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_CYP']
            else : 
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[0:48] if k in w_A if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_ HELLAS']
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[48:]  if k in w_A if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_CYP']
                

In [ ]:
for i in dfMM.index : ### GREECE
    for j in dfNN.index :
        if j in nE:
            if j not in n_A :
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[0:48] if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_ HELLAS']
            else :
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[0:48 ]if k in w_A if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_ HELLAS']

In [ ]:
for i in dfMM.index : ### CYPRUS
    for j in dfNN.index :
        if j in nC:
            if j not in n_A :
                model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[48:] if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_CYP']
            else :
                 model +=pulp.lpSum([HOURS[i,j,k]  for k in dfWW.index if k in dfWW.index[48:] if k in w_A if (i,j,k) in dftotal.index ])<=dfMM.loc[i,'CON_MON_CYP']                                                                                                                     

### CONSTRAINTS FOR TOTAL HOURS PER PERSON PER YEAR 

In [ ]:
for j in dfNN.index :  ### BOTH GREECE CYPRUS
    if j in nB:
        if j  in n_A :
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[0:12]   for k in dfWW.index if k in w_A  if (i,j,k) in dftotal.index ])>=0
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[12:24]  for k in dfWW.index if k in w_A if (i,j,k) in dftotal.index ])>=0
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[24:36]  for k in dfWW.index if k in w_A if (i,j,k) in dftotal.index ])>=0
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[36:]    for k in dfWW.index if k in w_A if (i,j,k) in dftotal.index ])>=0 

for j in dfNN.index :  ### BOTH GREECE CYPRUS
    if j in nB:
        if j not  in n_A :     
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[0:12]  for k in dfWW.index if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[0:12]  for k in dfWW.index if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[0:12]  for k in dfWW.index if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i  in dfMM.index[0:12]  for k in dfWW.index if (i,j,k) in dftotal.index ])<=1720 
        

In [ ]:
for j in dfNN.index :  ### GREECE      
    if j  in nE :
        if j in n_A :
            model +=pulp.lpSum([HOURS[i,j,k] for  i in dfMM.index[0:12]   for k in dfWW.index[0:48] if k in w_A  if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for  i in dfMM.index[12:24]  for k in dfWW.index[0:48] if k in w_A  if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for  i in dfMM.index[24:36]  for k in dfWW.index[0:48] if k in w_A  if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for  i in dfMM.index[36:]    for k in dfWW.index[0:48] if k in w_A  if (i,j,k) in dftotal.index ])<=1720  
        
for j in dfNN.index :  ### GREECE      
    if j  in nE :
        if j not in n_A :      
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[0:12]   for k in dfWW.index[0:48] if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[12:24]  for k in dfWW.index[0:48] if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[24:36]  for k in dfWW.index[0:48] if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[36:]    for k in dfWW.index[0:48] if (i,j,k) in dftotal.index ])<=1720

In [ ]:
for j in dfNN.index :  ### CYPRUS  
    if j  in nC :
        if j in n_A :
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[0:12]   for k in dfWW.index[48:] if k in w_A  if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[12:24]  for k in dfWW.index[48:] if k in w_A  if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[24:36]  for k in dfWW.index[48:] if k in w_A  if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[36:]    for k in dfWW.index[48:] if k in w_A  if (i,j,k) in dftotal.index ])<=1720 

for j in dfNN.index :  ### CYPRUS  
    if j  in nC :
        if j not in n_A :            
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[0:12]   for k in dfWW.index[48:] if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[12:24]  for k in dfWW.index[48:] if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[24:36]  for k in dfWW.index[48:] if (i,j,k) in dftotal.index ])<=1720
            model +=pulp.lpSum([HOURS[i,j,k] for i in dfMM.index[36:]    for k in dfWW.index[48:] if (i,j,k) in dftotal.index ])<=1720
        

### SOLVER

In [ ]:
model.solve()
plp.LpStatus[model.status]

### OUTPUT

In [ ]:
output = []
for i, j, k in HOURS:
    var_output = {
        'i': i,
        'j': j,
        'k': k,
        'HOURS': HOURS[(i, j, k)].varValue,
    }
    output.append(var_output)
output_df1 = pd.DataFrame.from_records(output).sort_values(['i', 'j','k'])
#output_df1.set_index(['i', 'j','k'], inplace=True)


In [ ]:
output_df1=output_df1[output_df1.HOURS !=0]
output_df1.shape

In [ ]:
print(output_df1.isna().any())
output_df1['HOURS'].sum()

In [ ]:
print( time.clock() - start_time, "seconds")

In [ ]:
output_df1=pd.pivot_table(output_df1, index=['j','k'], columns='i',  values='HOURS')
output_df2=output_df1.fillna(0)
output_df2

In [ ]:
output_df2.to_excel('outfinal_colab.xlsx')

In [ ]:
from google.colab import files
files.download('outfinal_colab.xlsx')